# Instagram Crawler
crawling for text, comments, and photos 

## 0. ready
1) Make sure you install Selenium, BeautifulSoup, and ChromeDriver. <br>



## 1. import packages

In [90]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup as bs

import time
import sys
from imp import reload

import re

from urllib.request import urlopen
import requests
import datetime 
import wget
import html2text

import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np

from random import *

## 2. login 

In [91]:
reload(sys)
#sys.setdefaultencoding('utf-8') python3에는 디폴트라 굳이 설정 ㄴㄴ

targetInstagramUrl = 'https://www.instagram.com/accounts/login/?source=auth_switcher'

options = webdriver.ChromeOptions()
#mobile_emulation = { "deviceName": "Nexus 5" }
#options.add_experimental_option("mobileEmulation", mobile_emulation)

login_id = input('put your login id : ')
login_pw = input('put your login password : ')

driver = webdriver.Chrome('chromedriver.exe', chrome_options=options)
driver.get(targetInstagramUrl)

#login process ----------------------------------------------------------------- start
#login process ----------------------------------------------------------------- start
time.sleep(2)

driver.find_elements_by_name("username")[0].send_keys(login_id)
driver.find_elements_by_name("password")[0].send_keys(login_pw)

driver.find_element_by_xpath("//form/div[4]/button").submit()

time.sleep(3)

driver.find_element_by_xpath("/html/body/div[4]/div/div/div[3]/button[2]").click()
#login process ----------------------------------------------------------------- end
#login process ----------------------------------------------------------------- end

put your login id : imhappy0530@naver.com
put your login password : norahvana13


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


## 3. search for hashtag

In [92]:
hashtag = input('What are you searching for? : ')
driver.get('https://www.instagram.com/explore/tags/'+ hashtag)

What are you searching for? : 고양이


## 4. Get links of each posts

In [118]:
def get_post_url(count = 100) : 

    links = []
    last_height = driver.execute_script("return document.body.scrollHeight") # Get scroll height

    while len(links) < count :
        
        ## 1. find links in page sources
        source = driver.page_source
        data = bs(source, 'html.parser')
        body = data.find('body')
        script = body.find('div')
        
        for link in script.findAll('a'):
            if re.match("/p", link.get('href')):
                links.append('https://www.instagram.com'+link.get('href'))

        links = list(set(links))
        
        ## 2. Scroll down to bottom        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        ## 3. Wait to load page
        time.sleep(0.5)

        ## 4. Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        #time.sleep(uniform(1.0, 2.0))

        ## 4 - 1. if all pages are already scrolled : break.
        if new_height == last_height:
            break
            
        #elif len(links) == count : 
        #    break
            
        ## 4 - 2. else : going on
        else : 
            last_height = new_height
    
    return links

In [119]:
#test
a = get_post_url(50)

In [120]:
len(a)

51

In [122]:
a

['https://www.instagram.com/p/B68J8CynDjc/',
 'https://www.instagram.com/p/B68J_TLAME3/',
 'https://www.instagram.com/p/B68KGUcl0my/',
 'https://www.instagram.com/p/B67nP0MnOjJ/',
 'https://www.instagram.com/p/B68J6HWn_EU/',
 'https://www.instagram.com/p/B668x6qnwm7/',
 'https://www.instagram.com/p/B68KIAGlH6V/',
 'https://www.instagram.com/p/B68KE4lJpK2/',
 'https://www.instagram.com/p/B68J2A7nlVs/',
 'https://www.instagram.com/p/B67ginanIdF/',
 'https://www.instagram.com/p/B68J7t5HvUh/',
 'https://www.instagram.com/p/B67quDXnkVc/',
 'https://www.instagram.com/p/B68KIltHzcO/',
 'https://www.instagram.com/p/B68KHNNgt3g/',
 'https://www.instagram.com/p/B68J7bcht8B/',
 'https://www.instagram.com/p/B67UboPHq0t/',
 'https://www.instagram.com/p/B68KIw_jstJ/',
 'https://www.instagram.com/p/B67zkJYHkF-/',
 'https://www.instagram.com/p/B68KAKApBFy/',
 'https://www.instagram.com/p/B68J0Aynp3k/',
 'https://www.instagram.com/p/B68J24RBhJh/',
 'https://www.instagram.com/p/B68KGVfAgc7/',
 'https://

## Get information from each link of link list

In [86]:
def info_from_post(link_list) : #get information from each link of link list
    result = pd.DataFrame()
    
    for i in range(len(link_list)) : 
        try:

            page = urlopen(link_list[i]).read()
            data = bs(page, 'html.parser')
            body = data.find('body')
            script = body.find('script')
            raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
            json_data=json.loads(raw)

            posts =json_data['entry_data']['PostPage'][0]['graphql']
            posts = json.dumps(posts)
            posts = json.loads(posts)

            x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
            x.columns =  x.columns.str.replace("shortcode_media.", "")
            result = result.append(x)

        except:
            np.nan
            
    result = result.drop_duplicates(subset = 'shortcode')
    result.index = range(len(result.index))
        
    return result


In [87]:
res = info_from_post(a)

In [88]:
res.head()

,__typename,accessibility_caption,caption_is_edited,comments_disabled,dash_info.is_dash_eligible,dash_info.number_of_qualities,dash_info.video_dash_manifest,dimensions.height,dimensions.width,display_resources,...,title,tracking_token,video_duration,video_url,video_view_count,viewer_can_reshare,viewer_has_liked,viewer_has_saved,viewer_has_saved_to_collection,viewer_in_photo_of_you
0,GraphImage,Image may contain: cat,False,False,NaN,NaN,NaN,1080,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
1,GraphImage,No photo description available.,False,False,NaN,NaN,NaN,1080,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
2,GraphImage,Image may contain: people sitting and indoor,True,False,NaN,NaN,NaN,1350,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
3,GraphSidecar,NaN,False,False,NaN,NaN,NaN,750,750,[{'src': 'https://scontent-icn1-1.cdninstagram...,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
4,GraphVideo,NaN,False,False,False,0.0,None,937,750,[{'src': 'https://scontent-icn1-1.cdninstagram...,...,,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,3.7,https://scontent-icn1-1.cdninstagram.com/v/t50...,10485.0,True,False,False,False,False


In [71]:
result.to_csv('savingname.csv', encoding = 'UTF-8-sig') #encoding for excel

In [70]:
result.head()

,__typename,accessibility_caption,caption_is_edited,comments_disabled,dimensions.height,dimensions.width,display_resources,display_url,edge_media_preview_comment.count,edge_media_preview_comment.edges,...,owner.requested_by_viewer,owner.username,shortcode,taken_at_timestamp,tracking_token,viewer_can_reshare,viewer_has_liked,viewer_has_saved,viewer_has_saved_to_collection,viewer_in_photo_of_you
0,GraphImage,"Image may contain: one or more people, people ...",False,False,1078,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,https://scontent-icn1-1.cdninstagram.com/v/t51...,27,"[{'node': {'id': '17931514555342332', 'text': ...",...,False,_icebear_eun,B67VOsBFiK6,1578203868,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,True,False,False,False,False
1,GraphSidecar,NaN,False,False,1080,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,https://scontent-icn1-1.cdninstagram.com/v/t51...,4,"[{'node': {'id': '17851755088776216', 'text': ...",...,False,leeds_daily,B67neY8An95,1578213434,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,True,False,False,False,False
2,GraphImage,"Image may contain: plant, outdoor and text",False,False,1350,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,https://scontent-icn1-1.cdninstagram.com/v/t51...,0,[],...,False,jeju_dongbaek_kitchen,B67X1hgAqGe,1578205235,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,True,False,False,False,False
3,GraphSidecar,NaN,False,False,1080,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,https://scontent-icn1-1.cdninstagram.com/v/t51...,19,"[{'node': {'id': '18019440139249680', 'text': ...",...,False,feeling1044,B67e17mn2uT,1578208908,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,True,False,False,False,False
4,GraphImage,"Image may contain: plant, table and indoor",False,False,1080,1080,[{'src': 'https://scontent-icn1-1.cdninstagram...,https://scontent-icn1-1.cdninstagram.com/v/t51...,93,"[{'node': {'id': '17851683964776150', 'text': ...",...,False,ojnuii,B663MT4nSxW,1578188120,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,True,False,False,False,False
